In [1]:

# Notebook must be in root in order to import app.db
from app.db import SessionLocal, Member
from app.visualize import _plot_exit_pie

from datetime import date, timedelta
import pandas as pd
import plotly.express as px

In [2]:
session = SessionLocal()

In [3]:
_plot_exit_pie(session, 90)

In [22]:
M = 360
days_back = 720
STEP = days_back//90 or 1
last = date.today() - timedelta(days=180)
first = last - timedelta(days=M + days_back)
exits = session.query(Member).filter((Member.date_of_exit>first) & (Member.date_of_exit<=last)).all()
big = pd.DataFrame()
for e in exits:
    big = big.append({
        'date':e.date_of_exit,
        'dest':e.exit_destination,
    }, ignore_index=True)

moving = pd.DataFrame()
for i in range(0, days_back, STEP):
    end = last - timedelta(days=i)
    start = end - timedelta(days=M)
    little = big[(big['date']>start) & (big['date']<=end)]
    breakdown = pd.DataFrame({dest:little[little['dest']==dest].shape[0]/little.shape[0] for dest in little['dest'].unique()}, index=[end])
    moving = moving.append(breakdown)
moving = moving.fillna(0)

fig = px.line(
    moving, 
    labels={'index':'date', 'value':'proportion'},
    title=f'{M}-Day Moving Averages',
    category_orders={'variable':
        ['Permanent Exit',
        'Temporary Exit',
        'Transitional Housing',
        'Emergency Shelter',
        'Unknown/Other']}
    )
print(STEP)
fig.show()

8


In [40]:
df = pd.DataFrame()
for e in exits:
    df = df.append({
        'date':e.date_of_exit,
        'dest':e.exit_destination,
    }, ignore_index=True)
print(df['date'].max() - df['date'].min())

90 days, 0:00:00


In [109]:
# # Iconic

# gbs = {dest:df[df['dest']==dest].groupby('date') for dest in df['dest'].unique()}
# wow = pd.DataFrame([pd.Series([len(g) for g in gbs[key].groups.values()], index=gbs[key].groups, name=key) for key in gbs]).T.sort_index()
# fig = px.line(wow, x=wow.index, y=[key for key in gbs])
# fig.show()

In [115]:
{dest:df[df['dest']==dest].shape[0]/df.shape[0] for dest in df['dest'].unique()}

{'Permanent Exit': 0.31343283582089554,
 'Emergency Shelter': 0.22388059701492538,
 'Temporary Exit': 0.19402985074626866,
 'Transitional Housing': 0.08955223880597014,
 'Unknown/Other': 0.1791044776119403}